In [ ]:
import numpy as np
import pandas as pd

# Load the data
data = np.load('exp_family/expo_family_ex_expo.npz')

model_probs = data['model_probs']  # shape: (100, 4, 3, 3)
distance_names = list(data['distance_names'])  # e.g., ['CvM', 'MMD', 'Wass', 'Stat']
model_labels = list(data['model_labels'])      # e.g., ['Expo', 'LogN', 'Gamma']
percentiles = data['percentiles']              # e.g., [0.1, 0.05, 0.01]

# You can change this if you want other percentiles
target_percentile_index = 2  # e.g., 0 for 0.1%, 1 for 0.05%, 2 for 0.01%

# Build a dictionary of probabilities per model per distance
records = []

for d_idx, dist_name in enumerate(distance_names):
    for model_idx, model_label in enumerate(model_labels):
        probs = model_probs[:, d_idx, target_percentile_index, model_idx]  # shape (n_observed,)
        for i, p in enumerate(probs):
            records.append({
                "Observed_ID": i,
                "Distance": dist_name,
                "Model": model_label,
                "Posterior_Probability": p
            })

# Convert to DataFrame
df = pd.DataFrame(records)

# Save to CSV
df.to_csv("expo_family_modelexpo_ABC_probabilities.csv", index=False)

In [ ]:
import numpy as np
import pandas as pd

# Load the data
data = np.load('exp_family/expo_family_ex_expo.npz')
theta_means = data['theta_means']     # shape: (n_obs, n_dist, n_perc, 3)
distance_names = list(data['distance_names'])  # e.g., ['CvM', 'MMD', 'Wass', 'Stat']
model_labels = list(data['model_labels'])      # e.g., ['Expo', 'LogN', 'Gamma']
percentiles = data['percentiles']              # e.g., [0.1, 0.05, 0.01]

# Choose which percentile to export (e.g., 0 for 0.1%)
target_percentile_index = 2
target_percentile_value = percentiles[target_percentile_index]

records = []

for obs_idx in range(theta_means.shape[0]):
    for d_idx, dist_name in enumerate(distance_names):
        for m_idx, model_label in enumerate(model_labels):
            record = {
                "Observed_ID": obs_idx,
                "Distance": dist_name,
                "Model": model_label,
                "Percentile": target_percentile_value,
                "Estimated_Theta": theta_means[obs_idx, d_idx, target_percentile_index, m_idx]
            }
            records.append(record)

# Create tidy DataFrame
df = pd.DataFrame(records)

# Save to CSV
df.to_csv("expo_family_modelexpo_ABC_thetaestimates.csv", index=False)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV
file_path = "expo_family_modelexpo_ABC_probabilities.csv"  
df = pd.read_csv(file_path)

# Check column names (optional)
print("Methods:", df.columns.tolist())

# Create boxplot
plt.figure(figsize=(10, 6))
plt.boxplot([df[col] for col in df.columns], labels=df.columns)

# Styling
plt.ylabel("Frequency of Model 1")
plt.title("Model Exp(1/2)")
plt.xticks(rotation=45)
plt.ylim(0, 1)
plt.grid(True)
plt.tight_layout()

# Save and show
plt.savefig("exp_family_expo_boxplots.png", dpi=300)
plt.show()